In [1]:
from dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv())

True

In [2]:
import faiss
from langchain_community.vectorstores import FAISS 
from langchain_ollama import OllamaEmbeddings
from langchain_community.docstore.in_memory import InMemoryDocstore

In [3]:
embeddings = OllamaEmbeddings(model='nomic-embed-text', base_url='http://localhost:11434')

In [12]:
db_path = "../13 - Vector Stores and Retrievals/health_supplements"
vector_store = FAISS.load_local(db_path,embeddings,allow_dangerous_deserialization=True)

In [13]:
question = "how to gain muscle mass?"

vector_store.search(query=question, k=5, search_type="similarity")

[Document(metadata={'source': './rag-dataset\\gym supplements\\2. High Prevalence of Supplement Intake.pdf', 'file_path': './rag-dataset\\gym supplements\\2. High Prevalence of Supplement Intake.pdf', 'page': 8, 'total_pages': 11, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'iLovePDF', 'creationDate': '', 'modDate': 'D:20241021113904Z', 'trapped': ''}, page_content='and strength gain among men. We detected more prevalent protein and creatine supplementation\namong younger compared to older ﬁtness center users, whereas the opposite was found for vitamin\nsupplementation. Other authors made similar observations [23] and it might reﬂect the diﬀerent\ntraining goals among age groups, i.e., more focus on strength and muscles among the younger and\nmore focus on health among the older age groups.\nComparable to other studies [4], we detected a positive correlation between training frequency\nand supplement usage. Nevertheless, the

In [33]:
retriever = vector_store.as_retriever(search_type="similarity",
                                      search_kwargs = {'k':3})

In [15]:
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002187C541900>, search_kwargs={'k': 3})

In [16]:
retriever.invoke(input=question)

[Document(metadata={'source': './rag-dataset\\gym supplements\\2. High Prevalence of Supplement Intake.pdf', 'file_path': './rag-dataset\\gym supplements\\2. High Prevalence of Supplement Intake.pdf', 'page': 8, 'total_pages': 11, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'iLovePDF', 'creationDate': '', 'modDate': 'D:20241021113904Z', 'trapped': ''}, page_content='and strength gain among men. We detected more prevalent protein and creatine supplementation\namong younger compared to older ﬁtness center users, whereas the opposite was found for vitamin\nsupplementation. Other authors made similar observations [23] and it might reﬂect the diﬀerent\ntraining goals among age groups, i.e., more focus on strength and muscles among the younger and\nmore focus on health among the older age groups.\nComparable to other studies [4], we detected a positive correlation between training frequency\nand supplement usage. Nevertheless, the

In [17]:
question = "how to lose weight?"
retriever.invoke(question)

[Document(metadata={'source': './rag-dataset\\health supplements\\1. dietary supplements - for whom.pdf', 'file_path': './rag-dataset\\health supplements\\1. dietary supplements - for whom.pdf', 'page': 12, 'total_pages': 17, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'iLovePDF', 'creationDate': '', 'modDate': 'D:20241021113754Z', 'trapped': ''}, page_content='Int. J. Environ. Res. Public Health 2021, 18, 8897\n13 of 24\n3.3. Dietary Supplements and Weight Loss\nThe number of people with excessive weight continues to rise, and ﬁghting obesity\nhas become one of the greatest challenges of contemporary medicine. A person wishing to\nlose weight needs to undertake several difﬁcult life-changes and practice them consistently\n(diet, physical activity, addiction-free). Meanwhile dietary supplements are presented as\na compelling alternative to traditional methods for combatting obesity. Wróbel-Harmas\net al. [107] demonstrated t

In [18]:
retriever = vector_store.as_retriever(search_type = 'similarity_score_threshold', 
                                      search_kwargs = {'k': 3, 'score_threshold': 0.1})

retriever.invoke(question)

[Document(metadata={'source': './rag-dataset\\health supplements\\1. dietary supplements - for whom.pdf', 'file_path': './rag-dataset\\health supplements\\1. dietary supplements - for whom.pdf', 'page': 12, 'total_pages': 17, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'iLovePDF', 'creationDate': '', 'modDate': 'D:20241021113754Z', 'trapped': ''}, page_content='Int. J. Environ. Res. Public Health 2021, 18, 8897\n13 of 24\n3.3. Dietary Supplements and Weight Loss\nThe number of people with excessive weight continues to rise, and ﬁghting obesity\nhas become one of the greatest challenges of contemporary medicine. A person wishing to\nlose weight needs to undertake several difﬁcult life-changes and practice them consistently\n(diet, physical activity, addiction-free). Meanwhile dietary supplements are presented as\na compelling alternative to traditional methods for combatting obesity. Wróbel-Harmas\net al. [107] demonstrated t

In [19]:
retriever = vector_store.as_retriever(search_type = 'mmr', 
                                      search_kwargs = {'k': 3, 'fetch_k': 20, 'lambda_mult': 1})

docs = retriever.invoke(question)
docs

[Document(metadata={'source': './rag-dataset\\health supplements\\1. dietary supplements - for whom.pdf', 'file_path': './rag-dataset\\health supplements\\1. dietary supplements - for whom.pdf', 'page': 12, 'total_pages': 17, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'iLovePDF', 'creationDate': '', 'modDate': 'D:20241021113754Z', 'trapped': ''}, page_content='Int. J. Environ. Res. Public Health 2021, 18, 8897\n13 of 24\n3.3. Dietary Supplements and Weight Loss\nThe number of people with excessive weight continues to rise, and ﬁghting obesity\nhas become one of the greatest challenges of contemporary medicine. A person wishing to\nlose weight needs to undertake several difﬁcult life-changes and practice them consistently\n(diet, physical activity, addiction-free). Meanwhile dietary supplements are presented as\na compelling alternative to traditional methods for combatting obesity. Wróbel-Harmas\net al. [107] demonstrated t

### RAG with LLAMA on OLLAMA

In [20]:
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain import hub

In [21]:
prompt = hub.pull("rlm/rag-prompt")
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [22]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [24]:
prompt = """
    You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question.
    If you don't know the answer, just say that you don't know.
    Answer in bullet points. Make sure your answer is relevant to the question and it is answered from the context only.
    Question: {question} 
    Context: {context} 
    Answer:
"""

prompt = ChatPromptTemplate.from_template(prompt)

In [25]:
llm = ChatOllama(model='llama3.2:3b', base_url='http://localhost:11434')

In [26]:
def format_docs(docs):
    return '\n\n'.join([doc.page_content for doc in docs])

context = format_docs(docs)

In [29]:
# print(context)

In [31]:
rag_chain = (
    {'context': retriever|format_docs, 'question': RunnablePassthrough()}
    |prompt
    |llm
    |StrOutputParser()
)

In [32]:
question = "how to lose weight?"

response = rag_chain.invoke(input=question)

print(response)

Here are the relevant points from the context related to losing weight:

• There is no evidence that using chromium or other popular ingredients in weight loss supplements promotes weight reduction.
• A 2013 meta-analysis of randomized studies found that chromium supplementation resulted in only a small additional weight reduction (0.5 kg or 0.75 kg).
• Consuming fat burners, especially those purchased online, has been compared to playing a game of "Russian roulette" due to the risk of serious side effects.
• It is not recommended to purchase dietary supplements from unauthorized buyers or increase the recommended dose in an attempt to accelerate weight loss.
• Using multiple products at the same time can lead to dangerously elevated concentrations of ingredients in the body.
• There is no evidence that any dietary supplement, including those with similar ingredients and concentrations, is effective for weight loss.
